In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip uninstall braindecode

In [2]:
!pip install braindecode==0.4.85
!pip install eeglib

     |████████████████████████████████| 324 kB 4.3 MB/s 
     |████████████████████████████████| 7.4 MB 20.7 MB/s 
  Created wheel for braindecode: filename=Braindecode-0.4.85-py3-none-any.whl size=71094 sha256=21718ccd24d36aef537f230925c1d0193cc5d06a9f9150231b15b5d2bee86427
  Stored in directory: /root/.cache/pip/wheels/eb/67/4c/81ad85409f913dd7e3ebb4fb5896839680c5627c04d91e3b78
Successfully built braindecode
     |████████████████████████████████| 1.7 MB 5.3 MB/s 


In [ ]:
# #读取test1.mat文件用mne
# import mne
# from braindecode.datasets.bbci import  BBCIDataset
# cnt1 = BBCIDataset(filename='/content/drive/MyDrive/EEG-dataset/high-gamma-dataset/a1.mat', load_sensor_names=None).load()
# # cnt1.plot()

In [ ]:
# cnt = BBCIDataset(filename='/content/drive/MyDrive/EEG-dataset/high-gamma-dataset/a1.mat', load_sensor_names=['C3', 'C4', 'Cz']).load()
# print(cnt)
# cnt.plot()

In [3]:
import logging
import sys
import os.path
from collections import OrderedDict
import numpy as np

from braindecode.datasets.bbci import  BBCIDataset
from braindecode.datautil.signalproc import highpass_cnt
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

def load_bbci_data(filename, low_cut_hz):
    load_sensor_names = ['C3', 'C4', 'Cz']
    loader = BBCIDataset(filename, load_sensor_names=load_sensor_names)

    cnt = loader.load()

    # Cleaning: First find all trials that have absolute microvolt values
    # larger than +- 800 inside them and remember them for removal later

    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                              ('Rest', [3]), ('Feet', [4])])
    clean_ival = [0, 4000]

    set_for_cleaning = create_signal_target_from_raw_mne(cnt, marker_def,
                                                  clean_ival)

    clean_trial_mask = np.max(np.abs(set_for_cleaning.X), axis=(1, 2)) < 800

    # now pick only sensors with C in their name
    # as they cover motor cortex
    C_sensors = load_sensor_names
    cnt = cnt.pick_channels(C_sensors)

    # Further preprocessings as descibed in paper
    cnt = resample_cnt(cnt, 250.0)
    cnt = mne_apply(
        lambda a: highpass_cnt(
            a, low_cut_hz, cnt.info['sfreq'], filt_order=3, axis=1),
        cnt)
    cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=1e-3,
                                                  init_block_size=1000,
                                                  eps=1e-4).T,
        cnt)

    # Trial interval, start at -500 already, since improved decoding for networks
    ival = [0, 2000]

    dataset = create_signal_target_from_raw_mne(cnt, marker_def, ival)
    dataset.X = dataset.X[clean_trial_mask]
    dataset.y = dataset.y[clean_trial_mask]
    return dataset

list_a = [] #存放所有.mat文件的路径
dir_str = '/content/drive/MyDrive/EEG-dataset/high-gamma-dataset'
for i in range(1, 29):
  list_a.append(os.path.join(dir_str, "a"+str(i)+".mat"))

datasets = []
for mat_path in list_a:
  datasets.append(load_bbci_data(mat_path, 0))



Creating RawArray with float64 data, n_channels=3, n_times=1225545
    Range : 0 ... 1225544 =      0.000 ...  2451.088 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/datasets/bbci.py:210: DeprecationWarning: events cannot be set directly. This warning will turn into an error after 0.24
  cnt.info["events"] = np.array(event_arr).T
This is not causal, uses future data....


Creating RawArray with float64 data, n_channels=3, n_times=612772
    Range : 0 ... 612771 =      0.000 ...  2451.084 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/mne_ext/signalproc.py:67: DeprecationWarning: sfreq cannot be set directly. Please use method inst.resample() instead. This warning will turn into an error after 0.24
  new_info["sfreq"] = new_fs


Creating RawArray with float64 data, n_channels=3, n_times=3131420
    Range : 0 ... 3131419 =      0.000 ...  6262.838 secs
Ready.


...

/usr/local/lib/python3.7/dist-packages/braindecode/mne_ext/signalproc.py:67: DeprecationWarning: sfreq cannot be set directly. Please use method inst.resample() instead. This warning will turn into an error after 0.24
  new_info["sfreq"] = new_fs


Creating RawArray with float64 data, n_channels=3, n_times=608505
    Range : 0 ... 608504 =      0.000 ...  1217.008 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/datasets/bbci.py:210: DeprecationWarning: events cannot be set directly. This warning will turn into an error after 0.24
  cnt.info["events"] = np.array(event_arr).T
This is not causal, uses future data....


Creating RawArray with float64 data, n_channels=3, n_times=304252
    Range : 0 ... 304251 =      0.000 ...  1217.004 secs
Ready.


/usr/local/lib/python3.7/dist-packages/braindecode/mne_ext/signalproc.py:67: DeprecationWarning: sfreq cannot be set directly. Please use method inst.resample() instead. This warning will turn into an error after 0.24
  new_info["sfreq"] = new_fs


In [4]:
import numpy as np
train_X, train_y = [], []
for D in datasets:
  train_X.append(np.transpose(np.asarray(D.X), [0, 2, 1]).reshape([-1, 25, 20, 3]))
  train_y.append(np.asarray(D.y))
train_X = np.concatenate(train_X, axis=0)
train_y = np.concatenate(train_y, axis=0)
train_X.shape

(13480, 25, 20, 3)

In [6]:
import random
sample_len = train_X.shape[0]
splite_ratio = 0.1
valid_num = int(sample_len*splite_ratio)
valid_idx = random.sample(list(range(sample_len)), valid_num)  #生成随机数，validation的下标们
train_idx = np.asarray(list(set(list(range(sample_len))) - set(list(valid_idx)))) #set集合可以来做减法，交并集

valid_X = train_X[valid_idx, :, :, :]
valid_y = train_y[valid_idx]
train_X = train_X[train_idx, :, :, :]
train_y = train_y[train_idx]

process data

baseline


In [7]:
## import tf and check gpu is_available
import tensorflow as tf
assert tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [8]:
## build model
def build_model(inputs):

   # CONV_1
  C_1 = tf.keras.layers.Conv2D(32, (3, 3), padding="SAME", name="C_1")(inputs)
  B_1 = tf.keras.layers.BatchNormalization(name="B_1")(C_1)
  L_1 = tf.keras.layers.LeakyReLU()(B_1)
  # D_1 = tf.keras.layers.Dropout(rate=0.1)(L_1)

  # CONV_2
  C_2 = tf.keras.layers.Conv2D(32, (3, 3), padding="SAME", name="C_2")(L_1)
  B_2 = tf.keras.layers.BatchNormalization(name="B_2")(C_2)
  L_2 = tf.keras.layers.LeakyReLU()(B_2)
  
  # MAX_POOLING_1
  M_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="SAME")(L_2) 
  print(M_1.shape) #13, 10, 32

  # CONV_3
  # C_3_IN = tf.concat((L_2, L_1), axis=3) 
  C_3 = tf.keras.layers.Conv2D(64, (3, 3), padding="SAME", name="C_3")(M_1)
  B_3 = tf.keras.layers.BatchNormalization(name="B_3")(C_3)
  L_3 = tf.keras.layers.LeakyReLU()(B_3)
  # D_3 = tf.keras.layers.Dropout(rate=0.1)(L_3)



  # CONV_4
  C_4 = tf.keras.layers.Conv2D(64, (3, 3), padding="SAME", name="C_4")(L_3)
  B_4 = tf.keras.layers.BatchNormalization(name="B_4")(C_4)
  L_4 = tf.keras.layers.LeakyReLU()(B_4)
  # D_4 = tf.keras.layers.Dropout(rate=0.1)(L_4)

  # MAX_POOLING_2
  M_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="SAME")(L_4) 
  print(M_2.shape)

  # FC
  F_IN = tf.reshape(M_2, [-1, 7 * 5 * 64])
  F_1 = tf.keras.layers.Dense(64, name="CNN_F_1")(F_IN)
  F_1_B = tf.keras.layers.BatchNormalization(name="CNN_F_1_B")(F_1)
  F_1_L = tf.keras.layers.LeakyReLU()(F_1_B) 
  F_1_D = tf.keras.layers.Dropout(rate=0.5)(F_1_L) 

  F_2 = tf.keras.layers.Dense(32, name="CNN_F_2")(F_1_D)
  F_2_B = tf.keras.layers.BatchNormalization(name="CNN_F_2_B")(F_2)
  F_2_L = tf.keras.layers.LeakyReLU()(F_2_B) 
  F_2_D = tf.keras.layers.Dropout(rate=0.5)(F_2_L) 

  F_3 = tf.keras.layers.Dense(4, name="CNN_F_3")(F_2_D)
  logits = tf.nn.softmax(F_3) #激活函数
  
  return logits
inputs = tf.keras.Input(shape=[25, 20, 3])
outputs = build_model(inputs)
model = tf.keras.Model(inputs, outputs)
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #将数字编码转化成one-hot编码格式，然后对one-hot编码格式的数据（真实标签值）与预测出的标签值使用交叉熵损失函数
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()] #检查y_true中的值（本身就是index）与y_pred中的最大值对应的index是否相等，针对稀疏情况的多分类
)

(None, 13, 10, 32)
(None, 7, 5, 64)


In [9]:
 ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/ckpt/high_gamma_pretrain_baseline/conv6net.h5",
    mointor="val_acc",
    save_best_only=True,
    save_weights_only=True,
    mode="max"
)
model.fit(
    x=train_X,
    y=train_y, 
    batch_size=16, #4.8.16.32
    epochs=50,  #20,30,,
    verbose="auto",
    validation_data=(valid_X, valid_y),
    callbacks=[ckpt],
    shuffle=True 
)

Epoch 1/50
759/759 [==============================] - 23s 12ms/step - loss: 1.2090 - sparse_categorical_accuracy: 0.4810 - val_loss: 0.9972 - val_sparse_categorical_accuracy: 0.5994
Epoch 2/50
759/759 [==============================] - 8s 10ms/step - loss: 1.0563 - sparse_categorical_accuracy: 0.5447 - val_loss: 0.9472 - val_sparse_categorical_accuracy: 0.5920
Epoch 3/50
759/759 [==============================] - 7s 10ms/step - loss: 1.0013 - sparse_categorical_accuracy: 0.5789 - val_loss: 0.8888 - val_sparse_categorical_accuracy: 0.6350
Epoch 4/50
759/759 [==============================] - 8s 11ms/step - loss: 0.9659 - sparse_categorical_accuracy: 0.5959 - val_loss: 0.8629 - val_sparse_categorical_accuracy: 0.6580
Epoch 5/50
759/759 [==============================] - 8s 10ms/step - loss: 0.9284 - sparse_categorical_accuracy: 0.6201 - val_loss: 0.8639 - val_sparse_categorical_accuracy: 0.6328
Epoch 6/50
759/759 [==============================] - 8s 10ms/step - loss: 0.8973 - sparse_cat